In [2]:
from nsepython import *   
import datetime

symbol = "NIFTY 50"
days = 20
end_date = datetime.datetime.now().strftime("%d-%b-%Y")
end_date = str(end_date)

start_date = (datetime.datetime.now()- datetime.timedelta(days=days)).strftime("%d-%b-%Y")
start_date = str(start_date)

df2=index_history("NIFTY 50",start_date,end_date)
df2["daily_change"]=df2["CLOSE"].astype(float) / df2["OPEN"].astype(float) #.pct_change()
df2=df2[['Index Name','HistoricalDate','daily_change', 'OPEN','HIGH','LOW', 'CLOSE']]
df2 = df2.iloc[1: , :]
print(df2)

   Index Name HistoricalDate  daily_change      OPEN      HIGH       LOW  \
1    Nifty 50    10 Nov 2022      0.999105  18044.35  18103.10  17969.40   
2    Nifty 50    09 Nov 2022      0.992823  18288.25  18296.40  18117.50   
3    Nifty 50    07 Nov 2022      0.999509  18211.75  18255.50  18064.75   
4    Nifty 50    04 Nov 2022      1.003531  18053.40  18135.10  18017.15   
5    Nifty 50    03 Nov 2022      1.004694  17968.35  18106.30  17959.20   
6    Nifty 50    02 Nov 2022      0.994771   18177.9  18178.75  18048.65   
7    Nifty 50    01 Nov 2022      1.000811   18130.7   18175.8  18060.15   
8    Nifty 50    31 Oct 2022      1.005695  17910.20  18022.80  17899.90   
9    Nifty 50    28 Oct 2022      1.001712  17756.40  17838.90  17723.70   
10   Nifty 50    27 Oct 2022      0.998061  17771.40  17783.90  17654.50   
11   Nifty 50    25 Oct 2022      0.991467  17808.30  17811.50  17637.00   
12   Nifty 50    24 Oct 2022      0.999684  17736.35  17777.55  17707.40   

       CLOS

In [3]:
nse_events()

,symbol,company,purpose,bm_desc,date
0,MAHAPEXLTD,Maha Rashtra Apex Corporation Limited,Financial results,Maha Rashtra Apex Corporation Limited has inf...,14-Nov-2022
1,AAREYDRUGS,Aarey Drugs & Pharmaceuticals Limited,Financial results,Aarey Drugs & Pharmaceuticals Limited has inf...,14-Nov-2022
2,AARVEEDEN,Aarvee Denims & Exports Limited,Financial results,Aarvee Denims & Exports Limited has informed ...,14-Nov-2022
3,AARVI,Aarvi Encon Limited,Financial results,Aarvi Encon Limited has informed the Exchange...,14-Nov-2022
4,ABHISHEK,Abhishek Corporation Limited,Financial results,Abhishek Corporation Limited has informed the...,14-Nov-2022
...,...,...,...,...,...
274,RAJESHEXPO,Rajesh Exports Limited,Financial results,Rajesh Exports Limited has informed the Excha...,15-Nov-2022
275,HINDZINC,Hindustan Zinc Limited,Dividend,Hindustan Zinc Limited has informed the Excha...,16-Nov-2022
276,JAYSREETEA,Jayshree Tea & Industries Limited,Other business matters,Jayshree Tea & Industries Limited has informed...,18-Nov-2022
277,WANBURY,Wanbury Limited,Financial results,Wanbury Limited has informed the Exchange abou...,28-Nov-2022


In [4]:
print(nse_fiidii())

    category         date  buyValue sellValue netValue
0     DII **  11-Nov-2022   9849.11   9233.57   615.54
1  FII/FPI *  11-Nov-2022  12533.98   8575.75  3958.23


In [5]:
print(nse_get_fno_lot_sizes()) # mode='pandas'

{'FINNIFTY': 40, 'MIDCPNIFTY': 75, 'BANKNIFTY': 25, 'NIFTY': 50, 'ABBOTINDIA': 40, 'ABFRL': 2600, 'ADANIENT': 250, 'ADANIPORTS': 625, 'AMARAJABAT': 1000, 'ABB': 250, 'AMBUJACEM': 1800, 'APOLLOHOSP': 125, 'APOLLOTYRE': 3500, 'ASHOKLEY': 5000, 'ASIANPAINT': 200, 'AUBANK': 1000, 'BAJFINANCE': 125, 'BANDHANBNK': 1800, 'BANKBARODA': 5850, 'BATAINDIA': 275, 'BEL': 5700, 'BHARTIARTL': 950, 'BHEL': 10500, 'CANFINHOME': 975, 'CHAMBLFERT': 1500, 'CHOLAFIN': 1250, 'CIPLA': 650, 'COALINDIA': 4200, 'COFORGE': 150, 'CONCOR': 1000, 'COROMANDEL': 700, 'ALKEM': 200, 'CROMPTON': 1500, 'CUB': 5000, 'DABUR': 1250, 'ABCAPITAL': 5400, 'DELTACORP': 2300, 'DIVISLAB': 150, 'DLF': 1650, 'DRREDDY': 125, 'EICHERMOT': 175, 'ESCORTS': 275, 'GAIL': 9150, 'GNFC': 1300, 'GODREJCP': 1000, 'GRANULES': 2000, 'ASTRAL': 275, 'GRASIM': 475, 'GUJGASLTD': 1250, 'HAL': 475, 'HCLTECH': 700, 'BAJAJFINSV': 500, 'HDFC': 300, 'HDFCAMC': 300, 'HEROMOTOCO': 300, 'HINDCOPPER': 4300, 'IDFC': 10000, 'IDFCFIRSTB': 15000, 'IGL': 1375, 'IN

In [6]:
# key = "NIFTY"
# key = "BANKNIFTY"
# key = "SME"
# key = "FO"
# key = "OTHERS"
# key = "ALL"
import pandas as pd

def nse_preopen(key="NIFTY"):
    returnTo = []
    payload = nsefetch("https://www.nseindia.com/api/market-data-pre-open?key="+key+"")   
    dataList = payload['data']
    for d in dataList:
      
        obj = {}
        obj['lastUpdateTime'] = d['detail']['preOpenMarket']['lastUpdateTime']
        obj['symbol'] = d['metadata']['symbol']
        obj['lastPrice'] = d['metadata']['lastPrice']
        obj['change'] = d['metadata']['change']
        obj['pChange'] = d['metadata']['pChange']
        obj['previousClose'] = d['metadata']['previousClose']
        obj['yearHigh'] = d['metadata']['yearHigh']
        obj['yearLow'] = d['metadata']['yearLow']
        returnTo.append(obj)
    return returnTo

nse_preopen("NIFTY")

[{'lastUpdateTime': '11-Nov-2022 09:07:18',
  'symbol': 'INFY',
  'lastPrice': 1546.65,
  'change': 44.700000000000045,
  'pChange': 2.976131029661443,
  'previousClose': 1501.95,
  'yearHigh': 1953.9,
  'yearLow': 1355},
 {'lastUpdateTime': '11-Nov-2022 09:07:19',
  'symbol': 'TECHM',
  'lastPrice': 1049,
  'change': 27.899999999999977,
  'pChange': 2.7323474684164113,
  'previousClose': 1021.1,
  'yearHigh': 1838,
  'yearLow': 943.7},
 {'lastUpdateTime': '11-Nov-2022 09:07:18',
  'symbol': 'BAJFINANCE',
  'lastPrice': 7149,
  'change': 173.89999999999964,
  'pChange': 2.4931542200111774,
  'previousClose': 6975.1,
  'yearHigh': 8045,
  'yearLow': 5220},
 {'lastUpdateTime': '11-Nov-2022 09:07:23',
  'symbol': 'WIPRO',
  'lastPrice': 398.5,
  'change': 9.600000000000023,
  'pChange': 2.4685008999742926,
  'previousClose': 388.9,
  'yearHigh': 726.8,
  'yearLow': 372.4},
 {'lastUpdateTime': '11-Nov-2022 09:07:19',
  'symbol': 'APOLLOHOSP',
  'lastPrice': 4386,
  'change': 100.5500000000

In [7]:

pre_open_df = pd.DataFrame(nse_preopen("NIFTY"))
pre_open_df

,lastUpdateTime,symbol,lastPrice,change,pChange,previousClose,yearHigh,yearLow
0,11-Nov-2022 09:07:18,INFY,1546.65,44.70,2.976131,1501.95,1953.90,1355.00
1,11-Nov-2022 09:07:19,TECHM,1049.00,27.90,2.732347,1021.10,1838.00,943.70
2,11-Nov-2022 09:07:18,BAJFINANCE,7149.00,173.90,2.493154,6975.10,8045.00,5220.00
3,11-Nov-2022 09:07:23,WIPRO,398.50,9.60,2.468501,388.90,726.80,372.40
4,11-Nov-2022 09:07:19,APOLLOHOSP,4386.00,100.55,2.346311,4285.45,5935.40,3361.55
5,11-Nov-2022 09:07:18,BAJAJFINSV,1740.00,39.65,2.331873,1700.35,1862.10,1072.72
6,11-Nov-2022 09:07:20,COALINDIA,258.80,5.20,2.050473,253.60,263.40,139.15
7,11-Nov-2022 09:07:19,TCS,3269.60,63.95,1.994915,3205.65,4043.00,2926.10
8,11-Nov-2022 09:07:18,SBIN,617.25,11.50,1.898473,605.75,622.70,425.00
9,11-Nov-2022 09:07:19,TATAMOTORS,420.00,7.80,1.892285,412.20,536.70,366.20


In [8]:
positions = nsefetch('https://www.nseindia.com/api/equity-stockIndices?index=SECURITIES%20IN%20F%26O')
# print(positions)
postions_df = pd.DataFrame(positions['data'])
postions_df.columns.values

postions_df[['symbol', 'identifier', 'series', 'open', 'dayHigh', 'dayLow',
       'lastPrice', 'previousClose', 'change', 'pChange', 'yearHigh', 'yearLow', 'nearWKH', 'nearWKL', 'perChange365d',
       'date365dAgo', 'chart365dPath', 'date30dAgo', 'perChange30d']]

,symbol,identifier,series,open,dayHigh,dayLow,lastPrice,previousClose,change,pChange,yearHigh,yearLow,nearWKH,nearWKL,perChange365d,date365dAgo,chart365dPath,date30dAgo,perChange30d
0,NAUKRI,,EQ,3909.95,4127.95,3900.1,4099.65,3815.15,284.50,7.46,6696,3313,38.774642,-23.744340,-45.93,14-Oct-2021,https://static.nseindia.com/sparklines/365d/NA...,16-Sep-2022,-8.06
1,COFORGE,,EQ,3755,3915,3755,3904.15,3676.1,228.05,6.20,6135,3210.05,36.362673,-21.622716,-32.85,14-Oct-2021,https://static.nseindia.com/sparklines/365d/CO...,16-Sep-2022,14.68
2,FSL,,EQ,104.8,109.5,104.5,108.65,102.5,6.15,6.00,206.65,93,47.423179,-16.827957,-51.82,14-Oct-2021,https://static.nseindia.com/sparklines/365d/FS...,16-Sep-2022,-3.14
3,HDFCBANK,,EQ,1545,1637,1538,1611.9,1524.75,87.15,5.72,1722.1,1271.6,6.399164,-26.761560,-14.27,14-Oct-2021,https://static.nseindia.com/sparklines/365d/HD...,16-Sep-2022,-3.09
4,HDFC,,EQ,2526,2688,2523.05,2647.25,2504.1,143.15,5.72,3021.1,2026,12.374632,-30.663870,-16.36,14-Oct-2021,https://static.nseindia.com/sparklines/365d/HD...,16-Sep-2022,-2.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,CONCOR,,EQ,809.95,813.95,765,770,803.3,-33.30,-4.15,828.75,554,7.088989,-38.989170,5.23,14-Oct-2021,https://static.nseindia.com/sparklines/365d/CO...,16-Sep-2022,-3.51
190,CHAMBLFERT,,EQ,303,303,285.15,286,298.8,-12.80,-4.28,516,260.8,44.573643,-9.662577,-14.53,14-Oct-2021,https://static.nseindia.com/sparklines/365d/CH...,16-Sep-2022,-4.17
191,EICHERMOT,,EQ,3730,3739,3501,3519,3700.85,-181.85,-4.91,3889.65,2159.55,9.529135,-62.950615,23.18,14-Oct-2021,https://static.nseindia.com/sparklines/365d/EI...,16-Sep-2022,-0.97
192,INDHOTEL,,EQ,349,349,308.65,317,338.7,-21.70,-6.41,349,171,9.169054,-85.380117,38.64,14-Oct-2021,https://static.nseindia.com/sparklines/365d/IN...,16-Sep-2022,-1.16


In [9]:
def nse_optionchain_ltp(payload,strikePrice,optionType,inp=0,intent=""):
    expiryDate=payload['records']['expiryDates'][inp]
    for x in range(len(payload['records']['data'])):
      if((payload['records']['data'][x]['strikePrice']==strikePrice) & (payload['records']['data'][x]['expiryDate']==expiryDate)):
          if(intent==""): 
              return payload['records']['data'][x][optionType]['lastPrice']
          if(intent=="sell"): 
              return payload['records']['data'][x][optionType]['bidprice']
          if(intent=="buy"): 
              return payload['records']['data'][x][optionType]['askPrice']
          
payload=nse_optionchain_scrapper('BANKNIFTY')
print(nse_optionchain_ltp(payload,41000,"CE",0,"sell"))

1222


In [10]:
print(nse_marketStatus())

{'marketState': [{'market': 'Capital Market', 'marketStatus': 'Close', 'tradeDate': '14-Nov-2022', 'index': 'NIFTY 50', 'last': 18349.7, 'variation': 321.5, 'percentChange': 1.78, 'marketStatusMessage': 'Market is Closed'}, {'market': 'Currency', 'marketStatus': 'Close', 'tradeDate': '14-Nov-2022', 'index': '', 'last': '', 'variation': '', 'percentChange': '', 'marketStatusMessage': 'Market is Closed'}, {'market': 'Commodity', 'marketStatus': 'Close', 'tradeDate': '14-Nov-2022', 'index': '', 'last': '', 'variation': '', 'percentChange': '', 'marketStatusMessage': 'Market is Closed'}, {'market': 'Debt', 'marketStatus': 'Close', 'tradeDate': '14-Nov-2022', 'index': '', 'last': '', 'variation': '', 'percentChange': '', 'marketStatusMessage': 'Market is Closed'}]}


In [11]:
print(nse_fiidii())


    category         date  buyValue sellValue netValue
0     DII **  11-Nov-2022   9849.11   9233.57   615.54
1  FII/FPI *  11-Nov-2022  12533.98   8575.75  3958.23


In [12]:
index_df = nse_index()
index_df = index_df[['timeVal', 'indexName', 'percChange',  'last', 'previousClose', 'high' , 'low', 'open', 'yearHigh', 'yearLow']]
index_df = index_df[((index_df['indexName'] == 'NIFTY 50') |
                      (index_df['indexName'] == 'NIFTY BANK') |
                     (index_df['indexName'] == 'NIFTY NEXT 50') | 
                    
                     (index_df['indexName'] == 'INDIA VIX')
                     )]
index_df

,timeVal,indexName,percChange,last,previousClose,high,low,open,yearHigh,yearLow
0,"Nov 11, 2022 15:33:20",NIFTY 50,1.78,"18,349.70","18,028.20","18,362.30","18,259.35","18,272.35","18,362.30","15,183.40"
1,"Nov 11, 2022 15:33:20",NIFTY NEXT 50,0.96,"43,189.00","42,778.10","43,329.25","43,033.75","43,224.20","45,509.70","35,018.20"
3,"Nov 11, 2022 15:33:20",NIFTY BANK,1.28,"42,137.05","41,603.75","42,345.50","41,918.10","42,163.40","42,345.50","32,155.35"
4,"Nov 11, 2022 15:33:20",INDIA VIX,-7.47,14.4050,15.5675,15.5675,14.2775,15.5675,33.9725,12.0800


In [13]:
nse_index()

,yearLow,last,indexName,yearHigh,previousClose,high,indexType,low,timeVal,percChange,indexOrder,open,indexSubType
0,"15,183.40","18,349.70",NIFTY 50,"18,362.30","18,028.20","18,362.30",eq,"18,259.35","Nov 11, 2022 15:33:20",1.78,0.00,"18,272.35",bm
1,"35,018.20","43,189.00",NIFTY NEXT 50,"45,509.70","42,778.10","43,329.25",eq,"43,033.75","Nov 11, 2022 15:33:20",0.96,1.00,"43,224.20",bm
2,"26,186.70","29,591.95",NIFTY IT,"39,446.70","28,505.75","29,733.15",eq,"29,163.75","Nov 11, 2022 15:33:20",3.81,2.00,"29,182.35",sc
3,"32,155.35","42,137.05",NIFTY BANK,"42,345.50","41,603.75","42,345.50",eq,"41,918.10","Nov 11, 2022 15:33:20",1.28,3.00,"42,163.40",sc
4,12.0800,14.4050,INDIA VIX,33.9725,15.5675,15.5675,eq,14.2775,"Nov 11, 2022 15:33:20",-7.47,4.0000,15.5675,bm
...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,-,1199.43,NIFTY BHARAT BOND INDEX - APRIL 2023,-,-,-,-,-,11-NOV-2022,-,-,-,-
105,-,1226.25,NIFTY BHARAT BOND INDEX - APRIL 2030,-,-,-,-,-,11-NOV-2022,-,-,-,-
106,-,1106.23,NIFTY BHARAT BOND INDEX - APRIL 2025,-,-,-,-,-,11-NOV-2022,-,-,-,-
107,-,1116.32,NIFTY BHARAT BOND INDEX - APRIL 2031,-,-,-,-,-,11-NOV-2022,-,-,-,-


In [14]:
print(nse_get_advances_declines())
print(nse_get_advances_declines().columns.values)


         symbol identifier series     open  dayHigh   dayLow lastPrice  \
0        NAUKRI                EQ  3909.95  4127.95   3900.1   4099.65   
1       COFORGE                EQ     3755     3915     3755   3904.15   
2           FSL                EQ    104.8    109.5    104.5    108.65   
3      HDFCBANK                EQ     1545     1637     1538    1611.9   
4          HDFC                EQ     2526     2688  2523.05   2647.25   
..          ...        ...    ...      ...      ...      ...       ...   
189      CONCOR                EQ   809.95   813.95      765       770   
190  CHAMBLFERT                EQ      303      303   285.15       286   
191   EICHERMOT                EQ     3730     3739     3501      3519   
192    INDHOTEL                EQ      349      349   308.65       317   
193        GNFC                EQ      647      656    590.4     603.3   

    previousClose  change  pChange  ...    nearWKH    nearWKL perChange365d  \
0         3815.15  284.50     7.

In [15]:
indiavix()
print(index_info("INDIA VIX"))


{'key': 'BROAD MARKET INDICES', 'index': 'INDIA VIX', 'indexSymbol': 'INDIA VIX', 'last': 14.405, 'variation': -1.16, 'percentChange': -7.46, 'open': 15.5675, 'high': 15.5675, 'low': 14.2775, 'previousClose': 15.5675, 'yearHigh': 33.9725, 'yearLow': 12.08, 'pe': '', 'pb': '', 'dy': '', 'perChange365d': 16.81, 'date365dAgo': '14-Oct-2021', 'chart365dPath': 'https://static.nseindia.com/sparklines/365d/INDIA-VIX.jpg', 'date30dAgo': '16-Sep-2022', 'perChange30d': -7.09, 'chart30dPath': 'https://static.nseindia.com/sparklines/30d/INDIA-VIX.jpg', 'chartTodayPath': 'https://static.nseindia.com/sparklines/today/INDIA-VIX.jpg', 'previousDay': 15.5675, 'oneWeekAgo': 15.655, 'oneMonthAgo': 20.175}


In [16]:
S0 = 34950.60
X = 35000.00
σ = 14.72
t = 3
call_theta,put_theta,call_premium,put_premium,call_delta,put_delta,gamma,vega,call_rho,put_rho=black_scholes_dexter(S0,X,t,σ="",r=10,q=0.0,td=365)
print(call_theta)
print(put_theta)
print(call_premium)
print(put_premium)
print(call_delta)
print(put_delta)
print(gamma)
print(vega)
print(call_rho)
print(put_rho)

-34.91893232086908
-25.33776939057332
172.00884682182732
192.65354240135275
0.4845700237993539
-0.5154299762006461
0.000872661908454388
12.621092329115475
1.3778633611216327
-1.4964855179670944


In [17]:
print(get_bhavcopy("04-06-2021"))


          SYMBOL  SERIES         DATE1   PREV_CLOSE   OPEN_PRICE   HIGH_PRICE  \
0     1018GS2026      GS   04-Jun-2021       106.00       102.00       102.00   
1      20MICRONS      EQ   04-Jun-2021        62.55        63.40        64.35   
2     21STCENMGM      EQ   04-Jun-2021        15.95        15.95        16.25   
3     3IINFOTECH      EQ   04-Jun-2021         8.95         8.90         9.00   
4        3MINDIA      EQ   04-Jun-2021     26261.80     26431.80     26648.70   
...          ...     ...           ...          ...          ...          ...   
2053   ZODJRDMKJ      BE   04-Jun-2021        36.60        37.40        37.40   
2054        ZOTA      EQ   04-Jun-2021       183.75       184.00       184.00   
2055       ZUARI      EQ   04-Jun-2021        97.30        97.85        98.40   
2056   ZUARIGLOB      EQ   04-Jun-2021        92.55        93.95        94.00   
2057   ZYDUSWELL      EQ   04-Jun-2021      2083.75      2095.25      2107.45   

       LOW_PRICE  LAST_PRIC

In [18]:
nse_preopen_movers(key="FO")


(        symbol     identifier  \
 0    MOTHERSON   MOTHERSONEQN   
 1     ASHOKLEY    ASHOKLEYEQN   
 2        BSOFT       BSOFTEQN   
 3     INDHOTEL    INDHOTELEQN   
 4         INFY        INFYEQN   
 5     TVSMOTOR    TVSMOTOREQN   
 6        TECHM       TECHMEQN   
 7   BAJFINANCE  BAJFINANCEEQN   
 8       NAUKRI      NAUKRIEQN   
 9     JUBLFOOD    JUBLFOODEQN   
 10       WIPRO       WIPROEQN   
 11   INTELLECT   INTELLECTEQN   
 12  APOLLOHOSP  APOLLOHOSPEQN   
 13  BAJAJFINSV  BAJAJFINSVEQN   
 14  PERSISTENT  PERSISTENTEQN   
 15       CANBK       CANBKEQN   
 16         PNB         PNBEQN   
 17         LTI         LTIEQN   
 18         FSL         FSLEQN   
 19      HONAUT      HONAUTEQN   
 20  TORNTPOWER  TORNTPOWEREQN   
 21     COFORGE     COFORGEEQN   
 22        LTTS        LTTSEQN   
 23   COALINDIA   COALINDIAEQN   
 24    MINDTREE    MINDTREEEQN   
 25  NATIONALUM  NATIONALUMEQN   
 26         TCS         TCSEQN   
 27  HINDCOPPER  HINDCOPPEREQN   
 28        BHE

In [19]:
movers=nse_preopen_movers("NIFTY")
# print(gainers)
print(movers)

(        symbol     identifier  \
0         INFY        INFYEQN   
1        TECHM       TECHMEQN   
2   BAJFINANCE  BAJFINANCEEQN   
3        WIPRO       WIPROEQN   
4   APOLLOHOSP  APOLLOHOSPEQN   
5   BAJAJFINSV  BAJAJFINSVEQN   
6    COALINDIA   COALINDIAEQN   
7          TCS         TCSEQN   
8         SBIN        SBINEQN   
9   TATAMOTORS  TATAMOTORSEQN   
10     HCLTECH     HCLTECHEQN   
11   ICICIBANK   ICICIBANKEQN   

                                              purpose  lastPrice  change  \
0               INTERIM DIVIDEND - RS 16.50 PER SHARE    1546.65   44.70   
1                  SPECIAL DIVIDEND - RS 18 PER SHARE    1049.00   27.90   
2                          DIVIDEND - RS 20 PER SHARE    7149.00  173.90   
3                              ANNUAL GENERAL MEETING     398.50    9.60   
4   ANNUAL GENERAL MEETING/DIVIDEND - RS 11.75 PER...    4386.00  100.55   
5   FACE VALUE SPLIT (SUB-DIVISION) - FROM RS 5/- ...    1740.00   39.65   
6                  INTERIM DIVIDEND -

In [20]:
nse_most_active(type="securities",sort="volume")
# type = "securities" (default)
# type = "etf"
# type = "sme"

# sort = "volume"
# sort = "value" (default)


,symbol,identifier,lastPrice,pChange,quantityTraded,totalTradedVolume,totalTradedValue,previousClose,exDate,purpose,yearHigh,yearLow,change,open,closePrice,dayHigh,dayLow,lastUpdateTime
0,ZOMATO,ZOMATOEQN,72.65,13.604378,397758848,397758848,2.807780e+10,63.95,None,None,169.00,40.60,8.70,66.00,72.80,73.55,65.50,11-Nov-2022 15:59:52
1,YESBANK,YESBANKEQN,16.85,2.121212,112993914,112993914,1.885868e+09,16.50,03-Jun-2019,ANNUAL GENERAL MEETING/ DIVIDEND - RS 2 PER S...,18.20,12.10,0.35,16.65,16.85,16.95,16.40,11-Nov-2022 15:59:55
2,IDEA,IDEAEQN,8.55,-1.156069,93250736,93250736,7.963613e+08,8.65,18-Aug-2022,ANNUAL GENERAL MEETING,16.80,7.75,-0.10,8.70,8.50,8.75,8.40,11-Nov-2022 15:59:54
3,TATASTEEL,TATASTEELEQN,107.60,2.769819,77933417,77933417,8.373946e+09,104.70,28-Jul-2022,FACE VALUE SPLIT (SUB-DIVISION) - FROM RS 10/-...,138.67,82.70,2.90,106.25,107.55,108.20,106.25,11-Nov-2022 15:59:54
4,PNB,PNBEQN,44.65,1.362089,76514774,76514774,3.418680e+09,44.05,22-Jun-2022,ANNUAL GENERAL MEETING/DIVIDEND - RS 0.64 PER ...,46.00,28.05,0.60,45.05,44.75,45.20,44.10,11-Nov-2022 15:59:50
5,SUZLON,SUZLONEQN,8.25,-0.602410,68794518,68794518,5.703066e+08,8.30,03-Oct-2022,RIGHTS 5:21 @ PREMIUM RS 3/-,12.02,5.41,-0.05,8.50,8.20,8.55,8.20,11-Nov-2022 15:59:53
6,NYKAA,NYKAAEQN,208.00,10.491368,46998979,46998979,9.828426e+09,188.25,10-Nov-2022,BONUS 5:1,428.95,162.50,19.75,192.15,207.85,224.45,188.25,11-Nov-2022 15:59:52
7,MOTHERSON,MOTHERSONEQN,71.00,0.780696,42473474,42473474,3.026235e+09,70.45,03-Oct-2022,BONUS 1:2,169.87,61.80,0.55,74.25,71.05,74.55,69.40,11-Nov-2022 15:59:59
8,UNIONBANK,UNIONBANKEQN,66.70,-0.224383,42272447,42272447,2.850854e+09,66.85,22-Jun-2022,ANNUAL GENERAL MEETING/DIVIDEND - RS 1.90 PER ...,69.35,33.50,-0.15,67.50,66.85,69.35,65.80,11-Nov-2022 15:59:59
9,RVNL,RVNLEQN,51.00,3.869654,39360410,39360410,2.001477e+09,49.10,22-Sep-2022,ANNUAL GENERAL MEETING/DIVIDEND - RS 0.25 PER...,54.15,29.00,1.90,52.25,51.15,52.25,49.35,11-Nov-2022 15:59:30


In [21]:
from nsepython import *  
from datetime import datetime
import pandas as pd

underlying_option_symbol = "BANKNIFTY" 
script_master = pd.read_csv('./scrip_master.csv')

# BANKNIFTY20OCT2232000PE
def getFNOToken(underlying_option_symbol, index, strike, option_type):
    underlying_option_expiry_list = expiry_list(underlying_option_symbol)
    date_object = datetime.strptime(underlying_option_expiry_list[index], '%d-%b-%Y')
    str_date = date_object.strftime('%d%b%Y').replace('20', '')
    str_date = str_date.upper()
    symbol_to = underlying_option_symbol+str_date+strike+option_type
    df = script_master[script_master['symbol'] == symbol_to]
    return df[['symbol', 'token', 'name', 'expiry', 'strike', 'lotsize']]
    
getFNOToken("BANKNIFTY", 0, "41500","CE")


C:\Users\tejal\AppData\Local\Temp\ipykernel_13188\912544131.py:6: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  script_master = pd.read_csv('./scrip_master.csv')


,symbol,token,name,expiry,strike,lotsize
64175,BANKNIFTY17NOV2241500CE,46794,BANKNIFTY,17NOV2022,4150000.0,25


In [22]:

from nsepython import *
def get_atm_strike(symbol):
    symbol = symbol.upper()
    option_chain_json = nse_optionchain_scrapper(symbol)
    data = option_chain_json['filtered']['data']
    ltp = data[0]['PE']['underlyingValue']
    strike_price_list = [x['strikePrice'] for x in data]
    atm_strike = sorted([[round(abs(ltp-i),2),i] for i in strike_price_list])[0][1]
    return atm_strike

def get_pe_ce_price(symbol,atm_strike):
    symbol = symbol.upper()
    option_chain = nse_optionchain_scrapper(symbol)
    for dictt in option_chain['filtered']['data']:
        if dictt['strikePrice'] ==atm_strike:
            pe_price = dictt['PE']['askPrice']
            ce_price = dictt['CE']['askPrice']
            return pe_price,ce_price
        
atm_strike = get_atm_strike("NIFTY")
get_pe_ce_price("NIFTY", atm_strike)

(86, 134.4)

In [24]:
oi_data, ltp, crontime = oi_chain_builder("NIFTY")
oi_data

C:\Users\tejal\AppData\Roaming\Python\Python310\site-packages\nsepython\rahu.py:147: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  oi_data = oi_data.append(oi_row, ignore_index=True)
C:\Users\tejal\AppData\Roaming\Python\Python310\site-packages\nsepython\rahu.py:147: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  oi_data = oi_data.append(oi_row, ignore_index=True)
C:\Users\tejal\AppData\Roaming\Python\Python310\site-packages\nsepython\rahu.py:147: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  oi_data = oi_data.append(oi_row, ignore_index=True)
C:\Users\tejal\AppData\Roaming\Python\Python310\site-packages\nsepython\rahu.py:147: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ve

,CALLS_Chart,CALLS_OI,CALLS_Chng in OI,CALLS_Volume,CALLS_IV,CALLS_LTP,CALLS_Net Chng,CALLS_Bid Qty,CALLS_Bid Price,CALLS_Ask Price,...,PUTS_Bid Price,PUTS_Ask Price,PUTS_Ask Qty,PUTS_Net Chng,PUTS_LTP,PUTS_IV,PUTS_Volume,PUTS_Chng in OI,PUTS_OI,PUTS_Chart
0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,1800.0,3451.75,3534.00,...,0.65,0.85,1500.0,0.05,0.65,56.47,3384.0,607.0,1335.0,0.0
1,0.0,0.0,0.0,0.0,0.00,0.00,0.00,1800.0,3403.30,3486.05,...,0.50,1.05,250.0,-8.85,1.05,58.23,369.0,72.0,72.0,0.0
2,0.0,0.0,0.0,0.0,0.00,0.00,0.00,1500.0,3332.85,3417.45,...,0.70,0.75,2150.0,-0.15,0.75,55.55,11516.0,1703.0,9148.0,0.0
3,0.0,0.0,0.0,0.0,0.00,0.00,0.00,1500.0,3242.85,3497.55,...,0.55,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.00,0.00,0.00,1800.0,3252.35,3373.95,...,0.75,1.00,200.0,-12.65,0.75,53.88,3.0,2.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,0.0,763.0,612.0,4789.0,21.18,0.65,-0.85,2650.0,0.65,0.80,...,1324.25,1388.70,250.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0
98,0.0,5191.0,2722.0,13162.0,22.33,0.80,-0.50,1350.0,0.70,0.80,...,1369.70,1461.90,1500.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0
99,0.0,672.0,279.0,1933.0,23.44,0.95,-0.35,7150.0,0.60,0.95,...,1414.95,1486.45,1800.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0
100,0.0,28511.0,6023.0,54766.0,23.16,0.65,-0.60,30850.0,0.65,0.70,...,1481.45,1559.80,1500.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0
